In [ ]:
# -*- coding: utf-8 -*- 
# This code is written in Python 3.7. To install the required packages, run the following commands:
# pip install pandas numpy matplotlib seaborn scikit-learn sympy
# This code is applicable to the Simargl 2022 dataset. 
# implemented Advanced Ensemble techniques: Bagging

import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load the cleaned Simargl 2022 dataset
df = pd.read_csv(
    '/home/ibibers@ads.iu.edu/IDS_Datasets/Combined_datasets/Simargl_cleaned_dataset.csv')

tf.keras.backend.clear_session()

In [ ]:
# Extract subsample of data
print(" Extract subsample of data: ")
print(df['ALERT'].value_counts())

In [ ]:
dropped_df = ['ALERT']
X = df.drop(dropped_df, axis=1)
y = df['ALERT']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

print(df['ALERT'].value_counts())

In [ ]:
# Initial train test split set and split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Convert data to TensorFlow tensors
# X_train_tf = tf.convert_to_tensor(X_train.values, dtype=tf.float32)
# y_train_tf = tf.convert_to_tensor(pd.factorize(y_train)[0], dtype=tf.int64)
# X_test_tf = tf.convert_to_tensor(X_test.values, dtype=tf.float32)

# Initialize MirroredStrategy for GPU acceleration
strategy = tf.distribute.MirroredStrategy()

# Train the models
with strategy.scope():
    # Define various base models
    base_models = [
        RandomForestClassifier(random_state=42),
        MLPClassifier(random_state=42),
        LogisticRegression(random_state=42),
        DecisionTreeClassifier(random_state=42)
    ]
    
    # Create a Bagging Classifier with diverse base models
    # Bagging is an ensemble technique that combines predictions from multiple models, and is effective in reducing overfitting.
    bagging_model = BaggingClassifier(
        base_models[0], n_estimators=len(base_models), random_state=42)

    # Train the bagging model
    bagging_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred_enc = bagging_model.predict(X_test)
    y_pred_enc_proba = bagging_model.predict_proba(X_test)

# Convert probabilities to TensorFlow tensors
pred_tf = tf.convert_to_tensor(y_pred_enc, dtype=tf.float32)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_enc)
print(f"Accuracy: {accuracy}")

In [ ]:
# Use inverse_transform to get original labels
y_pred = label_encoder.inverse_transform(y_pred_enc)
y_test_labels = label_encoder.inverse_transform(y_test)

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred))

In [ ]:

# Confusion matrix with string labels
conf_mat = confusion_matrix(y_test_labels, y_pred)

# Get unique class labels from y_test and y_pred_encoded
unique_labels = np.unique(np.concatenate(
    (y_test_labels, y_pred)))

# Plotting the Confusion Matrix with class labels
plt.figure(figsize=(6, 6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=unique_labels, yticklabels=unique_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Extract feature importances from base models
base_models_importances = {}

for i, model in enumerate(bagging_model.estimators_):
    if hasattr(model, 'feature_importances_'):
        base_models_importances[f'base_model_{i}'] = model.feature_importances_

# Create a DataFrame for visualization
importance_df = pd.DataFrame(base_models_importances, index=X.columns)

# Calculate average feature importance across base models
average_importance = importance_df.mean(axis=1)

# Sort features by average importance
sorted_importance = average_importance.sort_values(ascending=False)

# Visualize the feature importances
plt.figure(figsize=(14, 20))
sns.barplot(x=sorted_importance.values,
            y=sorted_importance.index, palette='mako')
plt.xlabel('Average Importance Value')
plt.ylabel('Feature Name')
plt.title('Average Feature Importance in BaggingClassifier')
plt.show()